# 01 - Ingestão de Dados (Camada Bronze)

## Fonte dos Dados

O conjunto de dados utilizado neste MVP foi obtido a partir da plataforma **Kaggle**, que disponibiliza bases de dados públicas para fins educacionais e de pesquisa.

O dataset [AI4I 2020 Predictive Maintenance Dataset](https://www.kaggle.com/datasets/stephanmatzka/predictive-maintenance-dataset-ai4i-2020)  refere-se a dados de manutenção de equipamentos industriais, contendo variáveis operacionais coletadas por sensores, bem como a indicação da ocorrência ou não de falhas.

## Processo de Coleta

A coleta dos dados foi realizada de forma manual, seguindo as etapas abaixo:

1. Download do conjunto de dados a partir da plataforma Kaggle
2. Exportação do dataset no formato CSV
3. Upload manual do arquivo CSV para o ambiente Databricks, utilizando o sistema de arquivos DBFS (Databricks File System)

Essa abordagem foi adotada por se tratar de um MVP com foco educacional, não sendo necessária a implementação de robôs de coleta ou integrações automatizadas com APIs externas.

## Objetivo da Camada Bronze

A camada Bronze tem como objetivo armazenar os dados em seu formato bruto, preservando integralmente as informações originais do dataset, sem aplicação de tratamentos, limpezas ou transformações.

Essa camada serve como base para as etapas posteriores de transformação (camada Silver) e modelagem analítica (camada Gold).


In [0]:
# Leitura da tabela RAW criada via interface gráfica do Databricks
df_raw = spark.table("default.ai_4_i_2020")

# Visualização inicial
df_raw.show(5)

# Verificação do schema
df_raw.printSchema()



+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+---------------+---+---+---+---+---+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Machine failure|TWF|HDF|PWF|OSF|RNF|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+---------------+---+---+---+---+---+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|              0|  0|  0|  0|  0|  0|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|              0|  0|  0|  0|  0|  0|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|              0|  0|  0|  0|  0|  0|
|  4|    L47183|   L|              298.2|                  308.6|                 

A renomeação das colunas foi realizada na camada Bronze para garantir compatibilidade com o Delta Lake e facilitar as transformações posteriores.

In [0]:
# Renomear as colunas para remover espaços, colchetes e outros caracteres inválidos
df_raw_renamed = df_raw \
    .withColumnRenamed("Product ID", "Product_ID") \
    .withColumnRenamed("Air temperature [K]", "Air_temperature_K") \
    .withColumnRenamed("Process temperature [K]", "Process_temperature_K") \
    .withColumnRenamed("Rotational speed [rpm]", "Rotational_speed_rpm") \
    .withColumnRenamed("Torque [Nm]", "Torque_Nm") \
    .withColumnRenamed("Tool wear [min]", "Tool_wear_min") \
    .withColumnRenamed("Machine failure", "Machine_failure") \
    .withColumnRenamed("TWF", "TWF") \
    .withColumnRenamed("HDF", "HDF") \
    .withColumnRenamed("PWF", "PWF") \
    .withColumnRenamed("OSF", "OSF") \
    .withColumnRenamed("RNF", "RNF")

# Escrita da camada Bronze como tabela Delta Gerenciada
df_raw_renamed.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("maintenance_bronze")


In [0]:
%sql
SELECT COUNT(*) FROM maintenance_bronze;

COUNT(*)
10000


A verificação confirmou que todo o conjunto de dados foi carregado com sucesso para a nuvem. O dataset é composto por 10.000 instâncias e a operação `COUNT(*)` demonstrou que todas as 10.000 observações foram corretamente persistidas na plataforma. Dessa forma, os dados encontram-se íntegros e aptos para prosseguir para a etapa de tratamento e curadoria na camada Silver.